# Use GPT4o to Create Labels for CCD v02

In [1]:
import pandas as pd
from tqdm import tqdm
# from gpt4all import GPT4All
from openai import OpenAI
import json
import re

# Load Unlabelled CCD

In [2]:
df = pd.read_parquet("CCD_Unlabeled.parquet")
df['product'] = ''
df['action'] = ''
df['sentiment'] = ''
print(df.shape)
display(df.head(3))

(7706, 6)


date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   

                                                text product action sentiment  
0                      i am still waiting on my card                           
1  what can i do if my card still hasnt arrived a...                           
2  i have been waiting over a week is the card st...

# Load OpenAI API Key

In [3]:
from pathlib import Path
from dotenv import load_dotenv
import os

cwd = Path.cwd()
env_path = cwd.parent.parent.parent
print(env_path)

load_dotenv(dotenv_path=env_path / '.env')
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key: Found")
else:
    print("API Key not found. Please ensure it is in the .env file.\n\n")

client = OpenAI()

D:\OneDrive\RChai\Documents\NUS_ISS_Prjs\4_PLP
OpenAI API Key: Found


# Product, Action and Sentiment Polarity List for this POC

In [4]:
pdt_l = ["savings account", "current account", "account", "loan", "fixed deposit", "forex account", "mobile app", "website", "atm", "credit card", "debit card", "card", "others"] 
action_l = ['apply', 'access', 'activate', 'block', 'cancel', 'close', 'deposit', 'dispute', 'earn', 'exchange', 'find', 'inquire', 'link' ,'open', 'pay', 'receive', 'redeem', 'refund', 'renew', 'report', 'reset', 'retrieve', 'schedule', 'select', 'transfer', 'unblock', 'update', 'unlink', 'verify', 'withdraw', 'unknown']
sentiment_l = ["very negative", "negative", "neutral", "positive", "very positive"]

# Get Labels from OpenAI

In [5]:
def make_prompt(customer_utterance: str, products: list, actions, sentiments: list):
    p1 = f"""Step 1, the bank has the following products in this product_list: {products}. """
    p2 = """Given a text, select the product from the product_list. If there is more than one product, select the product that is the focus of the text. """
    p3 = """If you do not know what is the product, select the last product in the product_list.  Your answer must be from the product_list."""
    
    p4 = f"""\nStep 2, the action_list is: {actions}. Select the action from the action_list that answers the question of what the user wanted to do with the product """
    p5 = """identified in step 1 at that point in time. If unsure or unable to, select the last action from the action_list. Your answer must be from the action_list. """
    
    p6 = f"""\nStep 3, the sentiment polarity has 5 levels: {sentiments}. Given the text, classify the sentiment. """
    
    p7 = """\nStep 4, give me the final answer in the format "product, action, sentiment". Do not give me anything else. Do not give any explanation. """
    
    return p1 + p2 + p3 + p4 + p5 + p6 + p7 + "\nThe TEXT is:"+ customer_utterance 

# prompt = make_prompt("i would like to dispute a direct debit payment", pdt_l, action_l, sentiment_l)
# prompt

In [6]:
import time
for i in tqdm(range(df.shape[0])):
    time.sleep(0.1)  # avoid rate limiter
    cust_utterance = df.loc[i, 'text'] 
    prompt = make_prompt(cust_utterance, pdt_l, action_l, sentiment_l)
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt}],
        temperature=0
    )
    i_lst = [i.lower().strip() for i in completion.choices[0].message.content.split(',')]
    product, action, sentiment = ["_".join(i.split()) if len(i.split()) > 1 else i for i in i_lst]
    df.loc[i, 'product'] = product
    df.loc[i, 'action'] = action
    df.loc[i, 'sentiment'] = sentiment

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 7706/7706 [1:52:52<00:00,  1.14it/s]


In [7]:
df.head()

date              dlg_id  \
0  2024-10-05  2024-10-07_1017--0   
1  2024-10-05  2024-10-07_1017--1   
2  2024-10-05  2024-10-07_1017--2   
3  2024-10-05  2024-10-07_1017--3   
4  2024-10-05  2024-10-07_1017--4   

                                                text product   action  \
0                      i am still waiting on my card    card  inquire   
1  what can i do if my card still hasnt arrived a...    card  inquire   
2  i have been waiting over a week is the card st...    card  inquire   
3  can i track my card while it is in the process...    card  inquire   
4  how do i know if i will get my card or if it i...    card  inquire   

  sentiment  
0  negative  
1  negative  
2  negative  
3   neutral  
4   neutral

In [8]:
df.tail()

date                 dlg_id  \
7701  2024-10-05  2024-10-07_1017--7701   
7702  2024-10-05  2024-10-07_1017--7702   
7703  2024-10-05  2024-10-07_1017--7703   
7704  2024-10-05  2024-10-07_1017--7704   
7705  2024-10-05  2024-10-07_1017--7705   

                                                   text           product  \
7701                         my checking account please  current__account   
7702            i see how about my savings account then  savings__account   
7703  hi could you check the balance of my savings a...  savings__account   
7704  i want to check the balance of my checking acc...  current__account   
7705                           check the savings please  savings__account   

       action sentiment  
7701  inquire   neutral  
7702  inquire   neutral  
7703  inquire   neutral  
7704  inquire   neutral  
7705  inquire   neutral

# Save Labelled CCD

In [9]:
# df.to_csv("ccd_labelled.csv", index=False)
# df.to_parquet("ccd_labelled.parquet", index=False)

In [82]:
df = pd.read_parquet("ccd_labelled.parquet")

# Do NOT Autorun the Below, they need to be manually checked and adjusted

# Check if LLM values need to be adjusted to fit aspect topics

In [83]:
def convert_double_underscore_to_space(item):
    tmp = item.split('__')
    if len(tmp) > 1:
        return " ".join(tmp)
    else:
        return tmp[0]

def convert_double_underscore_to_single_underscore(item):
    tmp = item.split('__')
    if len(tmp) > 1:
        return "_".join(tmp)
    else:
        return tmp[0]

def convert_single_underscore_to_space(item):
    tmp = item.split('_')
    if len(tmp) > 1:
        return " ".join(tmp)
    else:
        return tmp[0]

def convert_space_to_single_underscore(item):
    tmp = item.split()
    if len(tmp) > 1:
        return '_'.join(tmp)
    else:
        return tmp[0]

# convert_space_to_single_underscore('negative')

### Check 'sentiment' column values

In [84]:
df['sentiment'].unique()

array(['negative', 'neutral', 'very__negative', 'positive'], dtype=object)

In [85]:
df['sentiment'] = df['sentiment'].apply(lambda x: convert_double_underscore_to_space(x))
df['sentiment'].unique()

array(['negative', 'neutral', 'very negative', 'positive'], dtype=object)

### Check 'product' column values

In [86]:
df['product'] = df['product'].apply(lambda x: convert_double_underscore_to_single_underscore(x))
df['product'].unique()

array(['card', 'credit_card', 'mobile_app', 'debit_card', 'forex_account',
       'fixed_deposit', 'account', 'loan', 'atm', 'savings_account',
       'transfer', 'current_account', 'website', 'deposit',
       'cheque_deposit', 'others'], dtype=object)

In [87]:
bb = sorted(df['product'].unique())
aa = sorted([convert_space_to_single_underscore(item) for item in pdt_l])
for item in bb:
    if item not in aa:
        print(item)

cheque_deposit
deposit
transfer


##### fix "cheque_deposit"

In [88]:
fc = df['product'] == 'cheque_deposit'
df.loc[fc, :]

date                 dlg_id                                  text  \
1878  2024-10-05  2024-10-07_1017--1878  my cheque deposit didnt seem to work   

             product   action sentiment  
1878  cheque_deposit  deposit  negative

In [91]:
df.loc[1878, 'product'] = "savings_account"
df.loc[1878, :]

date                                   2024-10-05
dlg_id                      2024-10-07_1017--1878
text         my cheque deposit didnt seem to work
product                           savings_account
action                                    deposit
sentiment                                negative
Name: 1878, dtype: object

##### fix "deposit"

In [93]:
fc = df['product'] == 'deposit'
df.loc[fc, ['text', 'product']]

text  product
1810                why did my cash deposit not show up  deposit
1837  i have made a recent check and cash deposit th...  deposit
1864          my check or cash deposit has not shown up  deposit
5724  my cash deposit from a week ago hasnt shown up...  deposit

In [94]:
df.loc[1810, 'product'] = 'savings_account'
df.loc[1837, 'product'] = 'savings_account'
df.loc[1864, 'product'] = 'savings_account'
df.loc[5724, 'product'] = 'savings_account'

##### fix "transfer"

In [96]:
fc = df['product'] == 'transfer'
df.loc[fc, ['text', 'product']]

text   product
1208  could i please cancel my transfer i paid it in...  transfer
1211  ive made a horrible mistake  i transferred mon...  transfer
1216  i entered a typo when i was transferring money...  transfer
1218  it is extremely important that i cancel the tr...  transfer
1223  ive just made a money transfer to the wrong ac...  transfer
1226  i need urgent help i realize i sent my payment...  transfer
1252  i need help right now i realize i sent my paym...  transfer
1523  ive waited 2 days for a transfer to reach a re...  transfer
1532  i moved out of my old house two weeks ago and ...  transfer
1536  the day before yesterday i transferred funds t...  transfer
1537  how long does it take for transfers to be appr...  transfer
3658  can you help me transfer money into my friends...  transfer
6256  i have tried transferring money numerous times...  transfer
6268  hi  please help me as i have transferred some ...  transfer

In [98]:
df.loc[1208, 'product'] = 'account'
df.loc[1211, 'product'] = 'account'
df.loc[1216, 'product'] = 'account'
df.loc[1218, 'product'] = 'account'
df.loc[1223, 'product'] = 'account'
df.loc[1226, 'product'] = 'account'
df.loc[1252, 'product'] = 'account'
df.loc[1523, 'product'] = 'account'
df.loc[1532, 'product'] = 'account'
df.loc[1536, 'product'] = 'account'
df.loc[1537, 'product'] = 'account'
df.loc[3658, 'product'] = 'account'
df.loc[6256, 'product'] = 'account'
df.loc[6268, 'product'] = 'account'

In [99]:
bb = sorted(df['product'].unique())
aa = sorted([convert_space_to_single_underscore(item) for item in pdt_l])
for item in bb:
    if item not in aa:
        print(item)

### Check 'sentiment' column values¶

In [104]:
for item in action_l:
    print(item, end=", ")

apply, activate, block, cancel, close, deposit, dispute, earn, inquire, link, open, pay, redeem, report, reset, schedule, transfer, unblock, update, unlink, verify, withdraw, unknown, 

In [105]:
for item in sorted(df['action'].unique()):
    if item not in action_l:
        print(item, end=", ")

access, exchange, find, receive, refund, renew, retrieve, select, use, 

##### Investigate "access"

In [107]:
fc = df['action'] == 'access'
df.loc[fc, ['text', 'action']]

text  action
2849  i forgot my phone at the hotel so i cant acces...  access
2859  i have an emergency and lost my phone  i need ...  access
2891  my phone was left at a hotel and i cant access...  access
6021  if i was mugged and lost everything how do i a...  access
6024              left phone at hotel how to access app  access
6026  my phone is not with me at the moment so i wil...  access

##### Investigate "exchange"

In [111]:
fc = df['action'] == 'exchange'
for row in df.loc[fc, ['text', 'action']].itertuples():
    print(row.text, '\t--> ', row.action)

will you be able to exchange to eur 	-->  exchange
how can i exchange to eur 	-->  exchange
can i exchange my money for eur 	-->  exchange
can turn my money into another currency 	-->  exchange
instead of gbp how do i exchange for aud 	-->  exchange
i want to exchange usd and gbp with the app 	-->  exchange
how do i exchange currencies with this 	-->  exchange
i need australian dollars instead of uk currency 	-->  exchange
how can i exchange gbp for aud 	-->  exchange
can i use this app to change money 	-->  exchange
i traded some of my money from my native country into an atm to exchange for foreign currency and the money i received was a different equating amount 	-->  exchange
can i exchange to eur 	-->  exchange
how do i exchange my gbp to aud 	-->  exchange
i would like to exchange some currency 	-->  exchange
i received my salary in gbp how do i change this to my currency 	-->  exchange


##### Investigate "find"

In [113]:
fc = df['action'] == 'find'
for row in df.loc[fc, ['text', 'action']].itertuples():
    print(row.Index, row.text, '\t--> ', row.action)

216 i need to find the card i received in the app 	-->  find


##### Investigate "receive"

In [114]:
fc = df['action'] == 'receive'
for row in df.loc[fc, ['text', 'action']].itertuples():
    print(row.Index, row.text, '\t--> ', row.action)

4001 how to receive the actual card 	-->  receive


##### Investigate "refund"

In [115]:
fc = df['action'] == 'refund'
for row in df.loc[fc, ['text', 'action']].itertuples():
    print(row.Index, row.text, '\t--> ', row.action)

3414 i think my card payment has been refunded 	-->  refund


##### Investigate "renew"

In [116]:
fc = df['action'] == 'renew'
for row in df.loc[fc, ['text', 'action']].itertuples():
    print(row.Index, row.text, '\t--> ', row.action)

4868 how can my new card be renewed 	-->  renew


##### Investigate "retrieve"

In [119]:
fc = df['action'] == 'retrieve'
for row in df.loc[fc, ['text', 'product', 'action']].itertuples():
    print(row.Index, row.text, '\t--> ', row.product, ' | ', row.action)

3251 how do i get my card back quickly from a bank machine that it 	-->  atm  |  retrieve
3290 how do i get my card back after the atm swallowed it 	-->  atm  |  retrieve
6143 how do i retrieve my card from the machine 	-->  card  |  retrieve
6146 how do i get my card back after the atm has pulled it in 	-->  atm  |  retrieve
6148 is there a way i can get my atm card back from the machine 	-->  atm  |  retrieve
6170 how do i retrieve a trapped card from a atm 	-->  atm  |  retrieve


##### Investigate "select"

In [120]:
fc = df['action'] == 'select'
for row in df.loc[fc, ['text', 'product', 'action']].itertuples():
    print(row.Index, row.text, '\t--> ', row.product, ' | ', row.action)

2163 why can i not choose the card i want 	-->  card  |  select
7370 select my savings account 	-->  savings_account  |  select
7504 select my checking account 	-->  current_account  |  select


##### Investigate "use"

In [121]:
fc = df['action'] == 'use'
for row in df.loc[fc, ['text', 'product', 'action']].itertuples():
    print(row.Index, row.text, '\t--> ', row.product, ' | ', row.action)

785 i cant seem to be able to use my card 	-->  card  |  use
795 for a week now i have tried to use my card and it is still not working 	-->  card  |  use
805 i am unable to use my card for some reason 	-->  card  |  use
811 my card is not working when i try to use it 	-->  card  |  use
820 i tried using my card but it doesnt work 	-->  card  |  use
828 
i cant seem to be able to use my card


 	-->  card  |  use
873 my card doesnt work when i use it 	-->  card  |  use
2279 i tried using a virtual card for a hotel but it didnt work why not 	-->  card  |  use
2298 why cant i use a disposable card 	-->  card  |  use
2426 can i use my card at an atm 	-->  card  |  use
2675 tried using my atm card earlier at one of your locations for the first time  it didnt work  what is going on 	-->  atm  |  use
2689 i tried using my atm card at your notting hill location today  it did not work  am i doing something wrong 	-->  atm  |  use
2738 i tried to use the atm today but it would not accept my car

In [122]:
df.loc[fc, 'action'] = 'access'

In [124]:
fc = df['action'] == 'use'
for row in df.loc[fc, ['text', 'product', 'action']].itertuples():
    print(row.Index, row.text, '\t--> ', row.product, ' | ', row.action)

In [125]:
fc = df['action'] == 'access'
for row in df.loc[fc, ['text', 'product', 'action']].itertuples():
    print(row.Index, row.text, '\t--> ', row.product, ' | ', row.action)

785 i cant seem to be able to use my card 	-->  card  |  access
795 for a week now i have tried to use my card and it is still not working 	-->  card  |  access
805 i am unable to use my card for some reason 	-->  card  |  access
811 my card is not working when i try to use it 	-->  card  |  access
820 i tried using my card but it doesnt work 	-->  card  |  access
828 
i cant seem to be able to use my card


 	-->  card  |  access
873 my card doesnt work when i use it 	-->  card  |  access
2279 i tried using a virtual card for a hotel but it didnt work why not 	-->  card  |  access
2298 why cant i use a disposable card 	-->  card  |  access
2426 can i use my card at an atm 	-->  card  |  access
2675 tried using my atm card earlier at one of your locations for the first time  it didnt work  what is going on 	-->  atm  |  access
2689 i tried using my atm card at your notting hill location today  it did not work  am i doing something wrong 	-->  atm  |  access
2738 i tried to use the atm 

# View distribution

In [133]:
print(df['product'].value_counts(sort=True, ascending=False,  normalize=True).round(3))
print()
df['product'].value_counts(sort=True, ascending=False)

product
card               0.418
account            0.213
forex_account      0.103
atm                0.083
mobile_app         0.058
current_account    0.040
savings_account    0.039
credit_card        0.019
debit_card         0.019
loan               0.004
fixed_deposit      0.002
website            0.001
others             0.001
Name: proportion, dtype: float64



product
card               3223
account            1644
forex_account       797
atm                 638
mobile_app          449
current_account     306
savings_account     297
credit_card         149
debit_card          145
loan                 33
fixed_deposit        16
website               5
others                4
Name: count, dtype: int64

In [134]:
print(df['action'].value_counts(sort=True, ascending=False,  normalize=True).round(3))
print()
df['action'].value_counts(sort=True, ascending=False)

action
inquire     0.385
dispute     0.111
deposit     0.062
apply       0.060
transfer    0.060
withdraw    0.055
activate    0.031
report      0.030
pay         0.030
unknown     0.026
verify      0.023
open        0.020
close       0.019
link        0.018
update      0.016
block       0.014
cancel      0.012
unblock     0.011
access      0.006
reset       0.006
exchange    0.002
schedule    0.002
retrieve    0.001
select      0.000
find        0.000
refund      0.000
receive     0.000
renew       0.000
Name: proportion, dtype: float64



action
inquire     2968
dispute      854
deposit      481
apply        466
transfer     464
withdraw     425
activate     238
report       231
pay          229
unknown      204
verify       179
open         152
close        143
link         141
update       121
block        105
cancel        91
unblock       82
access        47
reset         45
exchange      15
schedule      12
retrieve       6
select         3
find           1
refund         1
receive        1
renew          1
Name: count, dtype: int64

In [135]:
print(df['sentiment'].value_counts(sort=True, ascending=False,  normalize=True).round(3))
print()
df['sentiment'].value_counts(sort=True, ascending=False)

sentiment
neutral          0.591
negative         0.399
very negative    0.007
positive         0.003
Name: proportion, dtype: float64



sentiment
neutral          4557
negative         3077
very negative      51
positive           21
Name: count, dtype: int64

# Save Adjusted Labelled Datasets

In [136]:
df.to_csv("ccd_labelled_adjusted.csv", index=False)
df.to_parquet("ccd_labelled_adjusted.parquet", index=False)